In [ ]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [ ]:
aoai_embedding_deployment_name=""
aoai_chat_deployment_name=""

In [ ]:
azdo_org_uri=""
azdo_pat=""
azdo_project=""

In [ ]:
from AzdoBacklogLoader import AzdoBacklogLoader
from AzdoPullRequestLoader import AzdoPullRequestLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

backlog_loader = AzdoBacklogLoader(azdo_org_uri, azdo_pat, azdo_project)
pr_loader = AzdoPullRequestLoader(azdo_org_uri, azdo_pat, azdo_project)
raw_docs = backlog_loader.load() + pr_loader.load()

chunked_docs = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100).split_documents(raw_docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment=aoai_embedding_deployment_name)

db = FAISS.from_documents(chunked_docs, embeddings)

In [ ]:
pr_retriever = db.as_retriever(
    search_kwargs={
        'k': 100,
        'fetch_k': 5000,
        'filter': {
            'type': 'pull_request'
        }
    }
)

comment_retriever = db.as_retriever(
    search_kwargs={
        'k': 100,
        'fetch_k': 5000,
        'filter': {
            'type': 'pull_request_comment'
        }
    }
)

commit_retriever = db.as_retriever(
    search_kwargs={
        'k': 100,
        'fetch_k': 5000,
        'filter': {
            'type': 'pull_request_commit'
        }
    }
)


In [ ]:
from langchain.chat_models.azure_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = AzureChatOpenAI(azure_deployment=aoai_chat_deployment_name, temperature=0)

In [ ]:
pr_template = """Satisfy the request based only on the following context:
{context}

Get the ids for the most recent {pr_count} pull requests completed. Return them as a CSV list.
"""

pr_prompt = ChatPromptTemplate.from_template(pr_template)

pr_chain = (
    {"context": pr_retriever, "pr_count": RunnablePassthrough()}
    | pr_prompt
    | model
    | StrOutputParser()
)

In [ ]:
comment_template = """Satisfy the request based only on the following context:
{context}

For each pull request id in [{ids}], get the list of related comment authors. Create a JSON object with each pull request id and related comment author list. Return all objects as a JSON array.
"""

comment_prompt = ChatPromptTemplate.from_template(comment_template)

comment_chain = (
    {"context": comment_retriever, "ids": pr_chain}
    | comment_prompt
    | model
    | StrOutputParser()
)

In [ ]:
import json

json.loads(comment_chain.invoke("5"))